In [74]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC

In [75]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set

In [76]:
train_set, test_set, val_set = read_data("data3", "train_data", "test_data", "valid_data")

Training set has 762 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [77]:
train_set_nb = sp.genfromtxt("data3/train_data.txt", delimiter="\t")
test_set_nb = test_set.values
val_set_nb = val_set.values

In [78]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [79]:
train_data = train_set_nb[~np.isnan(train_set_nb).any(axis=1)]

In [80]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_data, test_set_np, val_set_np)

Training set has 761 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [81]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [82]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [83]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [84]:
train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [85]:
def np_featuriser(train, test, val,feature_lsit):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [86]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [87]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.08:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

[  3.45901528e-03   9.43715652e-04   2.45842793e-02   2.19370568e-02
   4.63643829e-02   5.25611868e-02   2.00699275e-01   2.29780197e-01
   2.40903355e-01   8.79857809e-03   1.63272775e-03   7.73922378e-02
   2.57437639e-01   8.75295104e-02   1.41504560e-01   2.47296954e-01
   1.10267123e-01   2.40587521e-01   5.15644450e-02   3.82173977e-02
   2.89519767e-02   2.90464311e-03   7.86685640e-01   1.00000000e+00]
Most valuable features: 
6 0.200699275139 3PM
7 0.229780197251 3PA
8 0.240903355016 3P%
12 0.257437639275 OFF
13 0.0875295103744 DEF
14 0.141504559556 TRB
15 0.247296953898 AST
16 0.110267122541 STL
17 0.24058752131 BLK
23 0.786685640148 W
24 1.0 H
[0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20, 21, 22]


In [95]:
feature_list = [0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20, 21, 22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# Naive Bayes

In [89]:
clfNB = GaussianNB()

In [90]:
clfNB.fit(train_features_nb, train_labels)

GaussianNB()

In [93]:
predNB = clfNB.predict(test_features_nb)

# Accuracy Naive Bayes

In [94]:
accNB = metrics.accuracy_score(test_labels,predNB)
accNB
#72% all features

0.72834645669291342